In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


### MNIST con GradientBoosting
En esta segunda sesión práctica vamos a entrenar clasificadores basados en GradientBoosting.
Claramente estos clasificadores son más costosos de entrenar.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)

/home/joan/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [4]:
# Partición de los datos(7000.0)/(70000-7000)
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size = 0.2, random_state= 23)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state=23)



print(x_train.shape)
print(x_val.shape)
print(x_test.shape)



(56000, 784)
(7000, 784)
(7000, 784)


### **Ejercicio 2**   

Prueba un clasificador GradientBoosting

In [5]:
## GradientBoosting

gb = GradientBoostingClassifier()
gb.fit(x_train,y_train)
    
ypred = gb.predict(x_test)
acc = accuracy_score(y_test, ypred)
print(acc)



KeyboardInterrupt: 

Podrás comprobar como es un clasificador mucho más lento. 

### **Ejercicio 3**   

Para reducir el coste computacional se propone crear un pipeline donde se reduzca el número de características mediante PCA. En concreto el número de componentes (dimensiones) a las que reducimos con PCA es un hyperparámetro que tendrás que estimar con el conjunto de validación.

In [6]:
## Mejorar velocidad reduciendo dimensionalidad con PCAimp
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

for n in [2,4,8,16,32]:
    clf= make_pipeline(PCA(n_components=n), GradientBoostingClassifier())
    print(n)
    clf.fit(x_train, y_train)
    ypred = clf.predict(x_val)
    acc= accuracy_score(y_val, ypred)
    print(acc)

2
0.461
4


KeyboardInterrupt: 

De entre los diferentes parámeros que tiene el clasificador GradientBoosting de sklearn, cabría destacar:

**learning_rate**

**n_estimators**

**min_samples_split**

**max_features** 


Para más información leer la documentación en sklearn.

Alguno de estos parámetros influyen considerablemente en la velocidad de optimización. Por ejemplo **max_features** y **min_samples_split** entre otros.



### **Ejercicio 4**   

Se propone variar alguno de estos parámetros para ver si se obtiene una similar tasa de acierto pero con mejor velocidad.


In [7]:
# Calcular tiempo y tasa de acierto de la versión original con PCA 
n=16
clf = make_pipeline(PCA(n_components= n), GradientBoostingClassifier(max_features=6, n_estimators=100, min_samples_split=10))
print(n)
clf.fit(x_train, y_train)
ypred= clf.predict(x_val)
acc= accuracy_score(ypred, y_val)
print(acc)

16
0.9084285714285715


In [16]:
# Calcular tiempo y tasa de acierto modificando alguno de los parámetros propuestos

n=16
clf = make_pipeline(PCA(n_components=n), GradientBoostingClassifier(min_samples_split=20, max_features=10))
print(n)
clf.fit(x_train,y_train)
ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(acc)

16
0.8618571428571429


Finalmente sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma: el HistGradientBoostingClassifier. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



### **Ejercicio 5**   

Pruébalo y compara los tiempos de ejecución.

In [20]:
# Probar HistGradientBoostingClassifier 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting


for n in [2,4,8,16,32, 64, 128]:
    clf = make_pipeline(PCA(n_components=n), HistGradientBoostingClassifier())
    print(n)
    clf.fit(x_train,y_train)
    ypred = clf.predict(x_val)
    acc = accuracy_score(y_val, ypred)
    print(acc)

2
0.4642857142857143
4
0.6505714285714286
8
0.8957142857142857
16
0.9508571428571428
32
0.9652857142857143
64
0.9644285714285714
128
0.9615714285714285


In [29]:
# Probar HistGradientBoostingClassifier 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting


n=32
clf = make_pipeline(PCA(n_components=n), HistGradientBoostingClassifier(max_iter=1000, max_depth=10, max_bins=40))
print(n)
clf.fit(x_train,y_train)
ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(acc)

32
0.972


In [28]:
# Probar HistGradientBoostingClassifier 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
"""

LEARNING_RATE no superior  0,1
"""

n=32
clf = make_pipeline(PCA(n_components=n), HistGradientBoostingClassifier(max_iter=80, max_depth=10, max_bins=40))
print(n)
clf.fit(x_train,y_train)
ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(acc)

32
0.9614285714285714


In [ ]:
# Probar HistGradientBoostingClassifier 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting


n=32
clf = make_pipeline(PCA(n_components=n), HistGradientBoostingClassifier(max_iter=80, max_depth=10, max_bins=40))
print(n)
clf.fit(x_train,y_train)
ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(acc)

for mi in [10 20,40,60,80,10]:
    for md in [4, 6, 8, 10, 12]:
        

### **Ejercicio 6**   

Prueba los parámetros del HistGradientBoostingClassifier que mejoren la tasa de acierto. En cualquier caso la selección de estos parámetros debe seguir el protocolo de experimentación antes expuesto. Esto es, escoger el mejor parámetro con datos de validación y reportar resultados con los datos de test.